In [1]:
%load_ext autoreload
%autoreload 2


import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3,4"

import sys
import torch
import pandas as pd
from torch import nn
import numpy as np
from torch.optim.lr_scheduler import ExponentialLR
import wandb

sys.path.append("../NLP-DL-Project-hypo-to-hyper/pipeline_src/")


from config.config import TaskConfig
from train import CustomScheduler, train
from logger.logger import WanDBWriter
from trainer.train_epoch import train_epoch, predict
from metrics.metrics import get_all_metrics
from dataset.dataset import init_data
from logger.logger import WanDBWriter




if torch.cuda.is_available():
    device='cuda'
    print('GPU')
else:
    device='cpu'
    print('CPU')
    
    
SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
print(torch.cuda.device_count())

GPU
3


In [2]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    AutoConfig,
    AutoModelForCausalLM,
    LlamaTokenizer,
    LlamaForCausalLM

)

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict
    )


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 6.1
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('FILE')}
  warn(msg)
/srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/srv/home/rabikov/taxonomy_env/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')

In [3]:
config = TaskConfig()

config.batch_size = 32

config.data_path = "SemEval2018-Task9/training/data/1A.english.training.data.txt"
config.gold_path = "SemEval2018-Task9/training/gold/1A.english.training.gold.txt"
config.test_data_path = "SemEval2018-Task9/test/data/1A.english.test.data.txt"
config.test_gold_path = "SemEval2018-Task9/test/gold/1A.english.test.gold.txt"

config.device = device
config.using_peft = False
config.model_type = 'Auto' #Auto or Llama
config.wandb_log_dir = '/raid/rabikov/wandb/'
config.model_checkpoint = 'EleutherAI/gpt-neo-125m'
config.exp_name = config.model_checkpoint.replace('/', '-')
config.saving_path = '/raid/rabikov/model_checkpoints/' + config.exp_name

load_path = '/raid/rabikov/model_checkpoints/' + 'EleutherAI-gpt-neo-125m_epoch=1_MAP=0.06440927737634727.pth'


In [4]:
if config.model_type == 'Auto':
    model_type = AutoModelForCausalLM
    tokenizer_type = AutoTokenizer
elif config.model_type == "Llama":
    model_type = LlamaForCausalLM
    tokenizer_type = LlamaTokenizer

model = model_type.from_pretrained(config.model_checkpoint,
    #load_in_8bit=True,
    #torch_dtype=torch.float16,
    device_map="auto",)

tokenizer = tokenizer_type.from_pretrained(
    config.model_checkpoint,
    padding_side="left",
)


In [5]:
if config.using_peft:
    LORA_R = 8
    LORA_ALPHA = 16
    LORA_DROPOUT= 0.05
    LORA_TARGET_MODULES = [
        "q",
        "v",
    ]
    
    #model = prepare_model_for_int8_training(model)
    config_lora = LoraConfig(
        r=LORA_R,
        lora_alpha=LORA_ALPHA,
    # target_modules=LORA_TARGET_MODULES,
        lora_dropout=LORA_DROPOUT,
        bias="none",
        task_type="CAUSAL_LM",
    )
    model = get_peft_model(model, config_lora)
    model.print_trainable_parameters()


In [6]:
train_dataset, test_dataset, train_loader, val_loader = init_data(tokenizer, config)

In [7]:
checkpoint = torch.load(load_path)
model.load_state_dict(checkpoint['model'])
del checkpoint
torch.cuda.empty_cache()

In [8]:
config.gen_args = {"no_repeat_ngram_size":2,
                    "max_new_tokens":32, 
                    "num_return_sequences": 2,
                    "num_beams": 15,
                    "early_stopping":True, 
                    "num_beam_groups":5, 
                    "diversity_penalty":1.0,
                    "temperature": 0.9}


config.gen_args = {
            "no_repeat_ngram_size":2,
            "num_beams": 15,
            "early_stopping": True,
            "max_new_tokens": 16,
            "temperature": 0.9
        }

In [9]:
if config.using_peft:
    all_preds, all_labels = predict(model.model, tokenizer, val_loader, config)
else:
    all_preds, all_labels = predict(model, tokenizer, val_loader, config)

eval going: 100%|██████████| 46/46 [03:22<00:00,  4.40s/it]


In [10]:
metrics = get_all_metrics(all_labels, all_preds, limit=15)

In [11]:
metrics

{'MRR': 0.11209239130434782,
 'MAP': 0.040734578424795796,
 'P@1': 0.11209239130434782,
 'P@3': 0.04359148550724647,
 'P@5': 0.03497509057971022,
 'P@15': 0.03198060808454835}

In [19]:
metrics

{'MRR': 0.18615591397849462,
 'MAP': 0.07167302604802604,
 'P@1': 0.18615591397849462,
 'P@3': 0.0750448028673832,
 'P@5': 0.062141577060932075,
 'P@15': 0.05833561040214272}

In [17]:
all_preds[4], all_labels[4]

('person, actor, film director, cinematography, filmmaker, visual arts, person',
 'thrower, baseball player, jock, person')